[ref](https://classroom.udacity.com/courses/cs373/lessons/48704330/concepts/484805920923)

In [1]:
from math import *
import random

In [2]:
landmarks  = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
world_size = 100.0


class robot:
    def __init__(self):
        self.x = random.random() * world_size  # initialise with random
        self.y = random.random() * world_size
        self.orientation = random.random() * 2.0 * pi
        
        self.forward_noise = 0.0
        self.turn_noise    = 0.0
        self.sense_noise   = 0.0
    
    def set(self, new_x, new_y, new_orientation):
        if new_x < 0 or new_x >= world_size:
            raise ValueError('X coordinate out of bound')
        if new_y < 0 or new_y >= world_size:
            raise ValueError('Y coordinate out of bound')
        if new_orientation < 0 or new_orientation >= 2 * pi:
            raise ValueError('Orientation must be in [0..2pi]')
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)
    
    
    def set_noise(self, new_f_noise, new_t_noise, new_s_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.forward_noise = float(new_f_noise);
        self.turn_noise    = float(new_t_noise);
        self.sense_noise   = float(new_s_noise);
    
    
    def sense(self):
        Z = []
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            dist += random.gauss(0.0, self.sense_noise)
            Z.append(dist)
        return Z
    
    
    def move(self, turn, forward):
        if forward < 0:
            raise ValueError('Robot cant move backwards')
        
        # turn, and add randomness to the turning command
        orientation = self.orientation + float(turn) + random.gauss(0.0, self.turn_noise)
        orientation %= 2 * pi
        
        # move, and add randomness to the motion command
        dist = float(forward) + random.gauss(0.0, self.forward_noise)
        x = self.x + (cos(orientation) * dist)
        y = self.y + (sin(orientation) * dist)
        x %= world_size    # cyclic truncate
        y %= world_size
        
        # set particle
        res = robot()
        res.set(x, y, orientation)
        res.set_noise(self.forward_noise, self.turn_noise, self.sense_noise)
        return res
    
    
    def Gaussian(self, mu, sigma, x):
        # calculates the probability of x for 1-dim Gaussian with mean mu and var. sigma
        return exp(- ((mu - x) ** 2) / (sigma ** 2) / 2.0) / sqrt(2.0 * pi * (sigma ** 2))
    
    
    def measurement_prob(self, measurement):
        # calculates how likely a measurement should be
        prob = 1.0
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            prob *= self.Gaussian(dist, self.sense_noise, measurement[i])
        return prob
    
    
    def __repr__(self):
        return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), str(self.orientation))



def eval(r, p):
    s = 0.0;
    for i in range(len(p)): # calculate mean error
        dx = (p[i].x - r.x + (world_size/2.0)) % world_size - (world_size/2.0)
        dy = (p[i].y - r.y + (world_size/2.0)) % world_size - (world_size/2.0)
        err = sqrt(dx * dx + dy * dy)
        s += err
    return s / float(len(p))

In [30]:
forward_noise = 5.0 
turn_noise = 0.1
sense_noise = 5.0  # noise variance

myrobot = robot()

myrobot.set_noise(forward_noise, turn_noise, sense_noise)
myrobot.set(30., 50., pi/2)
myrobot = myrobot.move(-pi/2, 15.)

print(myrobot.sense())

myrobot = myrobot.move(-pi/2, 10.)

print(myrobot.sense())

[33.06686926692568, 39.3402893507033, 37.23534432750467, 34.18137281438475]
[39.212529957622685, 48.586856817416525, 41.54059854560854, 21.899392112217093]


## Particles
---
Initialise random robots in the world

In [38]:
N = 1000
p = []

for _ in range(N):
    x = robot()
    x.set_noise(0.05, 0.05, 5.0)  # add noise
    p.append(x)

p[:5]

[[x=89.958 y=44.058 orient=0.5153],
 [x=67.425 y=71.905 orient=3.0930],
 [x=11.044 y=60.100 orient=2.3018],
 [x=21.359 y=43.892 orient=4.0847],
 [x=6.6223 y=41.454 orient=0.8310]]

In [36]:
# turn 0.1 and move 5 meters
p = [rob.move(0.1, 5.0) for rob in p]

In [47]:
# actual location
myrobot = robot()
myrobot = myrobot.move(0.1, 5.0)
Z = myrobot.sense()

# initialise randomly guessed particles
N = 1000
p = []
for i in range(N):
    x = robot()
    x.set_noise(0.05, 0.05, 5.0)
    p.append(x)

p2 = []
for i in range(N):
    p2.append(p[i].move(0.1, 5.0))
p = p2

# given the particle's location, how likely measure it as Z
w = []

for rob in p:
    prob = rob.measurement_prob(Z)
    w.append(prob)

print(w[:5])

[1.1443973565120101e-05, 3.838024738302308e-30, 1.662745255731884e-07, 2.423425324124474e-50, 1.4743457538412627e-08]
